1. Pobierz dane wejściowe.

In [9]:
import time
import pandas as pd
import requests
from datetime import datetime, timedelta

# Oblicz pierwszy i ostatni dzień zeszłego miesiąca
today = datetime.today()
first_day_of_current_month = datetime(today.year, today.month, 1)
last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
first_day_of_last_month = datetime(last_day_of_last_month.year, last_day_of_last_month.month, 1)

# Konwertuj na UNIX timestamp
first_day = int(first_day_of_last_month.timestamp())
last_day = int(first_day_of_current_month.timestamp())

# Jeśli dzisiejszy dzień miesiąca jest większy niż 7, pobierz dane z bieżącego miesiąca
if today.day >= 7:
    first_day = int(first_day_of_current_month.timestamp())
    last_day = int((today + timedelta(days=1)).timestamp())

# Definicja nagłówków i danych do zapytania API
headers = {
    'X-BLToken': 'YOUR_TOKEN',
}
data = {
        'method': 'getInvoices',
}

# Pobierz wszystkie faktury
all_invoices = []
request = 0

while True:
    request += 1
    data['parameters'] = f'{{"date_from": {first_day}}}'
    response = requests.post('https://api.baselinker.com/connector.php', headers=headers, data=data)
    status = response.json().get('status')
    invoices = response.json().get('invoices', [])
    print(f'Request {request} status: {status}. Number of invoices: {len(invoices)}')
    
    if status == 'ERROR':
        print('Error! ')
        break

    if not invoices :
        print('No invoices?')
        break

    all_invoices.extend(invoices)
    first_day = invoices[-1]['date_add']
    if len(invoices) < 100 or first_day >= last_day:
        print('End of invoices.')
        break

    time.sleep(1)

# Zapisz dane do DataFrame
df = pd.DataFrame(all_invoices)


# Usuń niepotrzebne kolumny wg czasu
if 'date_add' in df.columns:
    df = df[df['date_add'] < last_day]

# Usuń duplikaty
df.drop_duplicates(subset=['invoice_id'], inplace=True)

Request 1 status: SUCCESS. Number of invoices: 100
Error! 


""


2. Sprawdź, czy dane są poprawne.

In [10]:
print('Zakres od dnia: ' + str(first_day_of_last_month))
print('Zakres do dnia: ' + str(first_day_of_current_month))
print('Liczba faktur: ' + str(len(df)))

df.info()

Zakres od dnia: 2024-08-01 00:00:00
Zakres do dnia: 2024-09-01 00:00:00
Liczba faktur: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


3. Zapisz dane do pliku XLSX

In [29]:
df.to_excel('invoices.xlsx', index=False)

Activate two-step verification on your Gmail account :
https://myaccount.google.com/u/1/security

Create an App Password :
https://myaccount.google.com/u/1/apppasswords

Tutorial: https://www.youtube.com/watch?v=sCsMfLf1MTg

In [6]:
import smtplib

fromaddr = 'raff1410x@gmail.com'  
toaddrs  = 'raff1410x@gmail.com'   
msg = 'Spam email Test'  

username = 'raff1410x'  
password = '' # Here

server = smtplib.SMTP('smtp.gmail.com', 587)  
server.ehlo()
server.starttls()
server.login(username, password)  
server.sendmail(fromaddr, toaddrs, msg)  
server.quit()

(221,
 b'2.0.0 closing connection 4fb4d7f45d1cf-5c3cc69ac64sm1549329a12.77 - gsmtp')

In [8]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email import encoders

# Email parameters
fromaddr = 'raff1410x@gmail.com'
toaddrs = 'raff1410x@gmail.com'
username = 'raff1410x'
password = ''  # Use environment variables for security

# Create the email message
msg = MIMEMultipart()
msg['From'] = fromaddr
msg['To'] = toaddrs
msg['Subject'] = "Faktury"

# Attach the body of the email
body = "Faktury z ostatniego miesiąca"
msg.attach(MIMEText(body, 'plain'))

# Attach the file
filename = "skoroszyt.xlsx"
with open("skoroszyt.xlsx", "rb") as attachment:
    part = MIMEBase('application', 'octet-stream')
    part.set_payload(attachment.read())
    encoders.encode_base64(part)
    part.add_header('Content-Disposition', f"attachment; filename= {filename}")
    msg.attach(part)

# Send the email
server = smtplib.SMTP('smtp.gmail.com', 587)
server.ehlo()
server.starttls()
server.login(username, password)
server.sendmail(fromaddr, toaddrs, msg.as_string())
server.quit()

print('Email sent')

Email sent
